In [ ]:
%pip install --upgrade --quiet  flashrank
# %pip install --upgrade --quiet  faiss

# OR  (depending on Python version)

%pip install --upgrade --quiet  faiss_cpu

Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Helper function for printing docs


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [
                f"Document {i + 1}:\n\n{d.page_content}\nMetadata: {d.metadata}"
                for i, d in enumerate(docs)
            ]
        )
    )

In [13]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [20]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

documents = TextLoader(
    "how_to/state.txt",
).load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
for idx, text in enumerate(texts):
    text.metadata["id"] = idx

embedding = OpenAIEmbeddings(model="text-embedding-ada-002")
retriever = FAISS.from_documents(texts, embedding).as_retriever(search_kwargs={"k": 20})

query = "What did the president say about American possibilities"
docs = retriever.invoke(query)
pretty_print_docs(docs)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Document 1:

of American possibilities, building an economy from the middle out and the bottom up, not the top down, investing in all America, in all Americans, to make sure everyone has a fair shot and we leave no one, no one behind. The pandemic no longer controls our lives. The vaccines that saved us from Covid are now being used to beat cancer. 5/26 https://www.nytimes.com/2024/03/08/us/politics/state-of-the-union-transcript-biden.html3/8/24, 6:43 PM Biden’s 2024 State of the Union Address: Read the
Metadata: {'source': 'how_to/state.txt', 'id': 23}
----------------------------------------------------------------------------------------------------
Document 2:

6:43 PM Biden’s 2024 State of the Union Address: Read the Full Transcript - The New York Times Turning setback into comeback — that’s what America does. That’s what America does. Folks, I inherited an economy that was on the brink. Now our economy is literally the envy of the world. Fifteen million new jobs in just three yea

###  Flash Ranker 

In [21]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import FlashrankRerank
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "What did the president say about American possibilities"
)
print([doc.metadata["id"] for doc in compressed_docs])

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[23, 38, 2]


In [22]:
pretty_print_docs(compressed_docs)

Document 1:

of American possibilities, building an economy from the middle out and the bottom up, not the top down, investing in all America, in all Americans, to make sure everyone has a fair shot and we leave no one, no one behind. The pandemic no longer controls our lives. The vaccines that saved us from Covid are now being used to beat cancer. 5/26 https://www.nytimes.com/2024/03/08/us/politics/state-of-the-union-transcript-biden.html3/8/24, 6:43 PM Biden’s 2024 State of the Union Address: Read the
Metadata: {'id': 23, 'relevance_score': np.float32(0.99813175), 'source': 'how_to/state.txt'}
----------------------------------------------------------------------------------------------------
Document 2:

It’s because of you that tonight we can proudly say the state of our union is strong and getting stronger. 8/26 https://www.nytimes.com/2024/03/08/us/politics/state-of-the-union-transcript-biden.html3/8/24, 6:43 PM Biden’s 2024 State of the Union Address: Read the Full Transcript - 

### QA re-ranking with Flashrank

In [23]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm, retriever=compression_retriever)

In [24]:
chain.invoke(query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'query': 'What did the president say about American possibilities',
 'result': 'The president mentioned building an economy from the middle out and the bottom up, investing in all Americans to ensure everyone has a fair shot, and leaving no one behind. He also talked about the future of possibilities that can be built together.'}